In [25]:
import numpy as np
import os

In [26]:
# read in the paths for every .txt file
filepaths = []
for root, dirs, files in os.walk(os.getcwd()+"/review_polarity_full/txt_sentoken/"):
    for file in files:
        if os.path.splitext(file)[1] == '.txt':
                filepaths.append(os.path.join(root, file))

In [27]:
X_raw = list()
y = list()

for filepath in filepaths:
    
    if "pos" in filepath:
        with open(filepath, "r") as f:
            X_raw.append(f.read())
        y.append(1)

    if "neg" in filepath:
        with open(filepath, "r") as f:
            X_raw.append(f.read())
        y.append(-1)
y = np.array(y)

In [28]:
all_words = []
words_per_file = []
for review in X_raw:
    words = []
    tokens = review.split(" ")
    for token in tokens:
        all_words.append(token.strip("\n"))
        words.append(token.strip("\n"))
    words_per_file.append(words)

In [29]:
vocab = sorted(list(set(all_words)))
len(vocab)

50921

In [13]:
X = []
for review in words_per_file:
    bow_vector = np.zeros(len(vocab))
    for word in review:
        for i, voc in enumerate(vocab):
            if voc == word:
                bow_vector[i] = 1
    X.append(bow_vector)
X = np.array(X)
np.savetxt('X_matrix.txt', X, fmt='%d')

In [6]:
X = np.loadtxt("X_matrix.txt")

print(np.shape(X))

(2000, 50921)


In [1]:
X = X.T
print(np.shape(X))
print(np.shape(X[0]))

NameError: name 'X' is not defined

1


In [153]:
class GDClassifier:

    def __init__(self, learning_rate=0.1, epoch=5, batch=100, l=0.001):
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.batch = batch
        self.l = l
        
    def fit(self, X, y):
        w = np.random.normal(0, 1, (np.size(X, 0), 1))
        X_train = X[:, 0:int(np.size(X, 1) * 0.8)]
        y_train = y[0:int(np.size(X, 1) * 0.8)]

        X_test = X[:, int(np.size(X, 1) * 0.8):]
        y_test = y[int(np.size(X, 1) * 0.8):]

        for i in range(self.epoch):
            for j in range(np.size(X_train, 1)//self.batch):
                j_start = self.batch * j
                j_end = self.batch * (j+1)
                X_batch = X[:, j_start:j_end]
                y_batch = y[j_start:j_end]

                grad_w = self._compute_gradient(X_batch, y_batch, w, self.l)
                w -= self.learning_rate * grad_w

            train_loss = self._loss(X_train, y_train, w)
            test_acc = self.score(X_test, y_test, w)
            print(f"Train loss is {train_loss} at epoch {i+1}")
            print(f"Test accuracy is {test_acc} at epoch {i+1}")
            print()

    def predict(self, X, w):
        result = np.zeros(np.size(X, 1))
        for i, product in enumerate(w.T @ X):
            if product > 0:
                result[i] = 1
            elif product < 0:
                result[i] = -1
        return result

    def score(self, X, y, w):
        predicted = self.predict(w, X)
        correct = 0
        for i, sentiment in enumerate(y):
            if predicted[i] == sentiment:
                correct += 1

        return (correct / len(y)) * 100

    def _loss(self, X, y, w):
        result = 0
        for i in range(np.size(X, 1)):
            result += max(0, 1 - y[i] * w.T @ X[:, i])
        return 0.5 * pow(np.linalg.norm(w), 2) + result

    def _compute_gradient(self, X, y, w, l):
        sum = 0
        for i in range(np.size(X, 1)):
            if y[i] * w.T @ X[:, i] >= 1:
                sum += 0
            else:
                sum += -y[i] * X[:, i]
        return l * w + sum
                


In [ ]:
model = GDClassifier()
model.fit(X, y)